# Objective: Fine-tune GPT-2 with the resulting filtered dataset

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2TokenizerFast, DataCollatorWithPadding, GPT2ForSequenceClassification, set_seed
import torch
from torch.nn.functional import one_hot
from utils_ import tokenize, train_classifier
import pickle
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

In [2]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [3]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [4]:
# read in indices removed by AFLite for each seed
with open('removed_idx.pkl', 'rb') as f:
    removed = pickle.load(f)

## 2. Pre-processing Routine for SNLI
- Get SNLI Dataset (Train fold) and shuffle it using the same seed as used for obtaining GPT-2 based Feature Representation (see notebook [Filtering_Part1.ipynb](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-small/notebooks_and_scripts/Filtering_Part1.ipynb))
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Partition data 10%/90%; use the 90% as unfiltered `train`
- Filter out what AFLite run removed (see notebook [Filtering_Part2.ipynb](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-small/notebooks_and_scripts/Filtering_Part2.ipynb))
- Tokenise

In [5]:
def preprocess_snli(seed):
    
    # set up `train` dataset
    snli_train = load_dataset('snli', split = 'train').shuffle(seed = 42)
    snli_train = snli_train.filter(lambda x: x['label'] != -1).map( \
        lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
        batched = True)
    train = snli_train.select(range(int(len(snli_train)/10), len(snli_train)))
    
    # filter out what AFLite run removed
    removed_idx = [int(x) for x in removed[seed].split(',')[1:]] 
    train = train.select(set(range(len(train))) - set(removed_idx))
    
    # tokenize data
    train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
    len_bef_exclusion = len(train)

    # exclude instances with > 128 tokens
    train = train.filter(lambda x: x['exclude'] == False)
    len_aft_exclusion = len(train)

    # print message if instances were in fact excluded
    if len_bef_exclusion - len_aft_exclusion > 0:

        print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
              f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')

    # keep only needed columns, set data format to PyTorch
    train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

    return(train)

### 3. Run fine-tuning step and save resulting models

In [6]:
# hyper-parameters for model training
batch_size = 64 # constrained by GPU memory
lr = 1e-5 # set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108

# from Filtering_Part2.ipynb
AFLite_seeds = [0, 1, 2, 3, 4]
    
# Begin fine-tuning
for seed in AFLite_seeds:
    
    # print log message
    print(f'Seed {seed} begin')
    
    # read in filtered data
    data = preprocess_snli(seed)
    print('Filtered Data Read In')
    
    # instantiate new GPT-2 based model
    model = GPT2ForSequenceClassification.from_pretrained("gpt2", 
                                      num_labels=3,
                                      problem_type="multi_label_classification")
    model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer
    
    # set up data loader
    dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, \
                                             shuffle=True, collate_fn=data_collator)
    
    # set up optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)    
    
    # fine-tune model
    torch.save(train_classifier(model, dataloader, optimizer, device), \
              'AFLite_fine_tuned_model_seed' + str(seed) + '.pth')
    
    # print log message
    print(f'Seed {seed} complete')    

Seed 0 begin


Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

  0%|          | 0/262396 [00:00<?, ?ex/s]

...

loss: 1.528919  [    0/265398]
loss: 0.629002  [26496/265398]
loss: 0.644122  [52992/265398]
loss: 0.521323  [79488/265398]
loss: 0.615192  [105984/265398]
loss: 0.603087  [132480/265398]
loss: 0.637906  [158976/265398]
loss: 0.478359  [185472/265398]
loss: 0.548402  [211968/265398]
loss: 0.481871  [238464/265398]
loss: 0.448869  [264960/265398]
Epoch average loss: 0.5331658720970154


  0%|          | 0/4147 [00:00<?, ?it/s]

loss: 0.430480  [    0/265398]
loss: 0.381909  [26496/265398]
loss: 0.373069  [52992/265398]
loss: 0.370104  [79488/265398]
loss: 0.423201  [105984/265398]
loss: 0.445478  [132480/265398]
loss: 0.397075  [158976/265398]
loss: 0.329071  [185472/265398]
loss: 0.320523  [211968/265398]
loss: 0.317310  [238464/265398]
loss: 0.354632  [264960/265398]
Epoch average loss: 0.38367968797683716


  0%|          | 0/4147 [00:00<?, ?it/s]

loss: 0.354580  [    0/265398]
loss: 0.364377  [26496/265398]
loss: 0.302272  [52992/265398]
loss: 0.432563  [79488/265398]
loss: 0.349704  [105984/265398]
loss: 0.283945  [132480/265398]
loss: 0.290156  [158976/265398]
loss: 0.301987  [185472/265398]
loss: 0.365680  [211968/265398]
loss: 0.317792  [238464/265398]
loss: 0.377326  [264960/265398]
Epoch average loss: 0.3356095552444458
Done!
Seed 4 complete
